Importing dependencies

In [2]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from pandas import read_csv
import os


# Election Results

## Loading Data

**VEST Data**, which included merged election and shapefile data, was found on [Harvard's Dataverse](https://dataverse.harvard.edu/dataverse/electionscience) 
In order to download the data, navigate to **2018 Precinct-Level Election Results**, then locate *wa_2018.zip*, select *Access File* and click on **Download** 
The file will be a zip folder *wa_2018*, which will contain the *wa_2018.shp*

**WA 2018 election results** data on a precinct level is stored on the [secretary of state's website](https://www.sos.wa.gov/elections/research/election-results-and-voters-pamphlets.aspx) 
In order to download the data, navigate to **2018 Elections** then click on **Results** within the *General* tab, select **Export Results** on the new page, right click on *.csv* icon within *All County Precincts (participating counties)* and "Save as..." *.csv*
The file will be saved as *20181106_allstateprecincts.csv*

As stated  in the *documentation.txt* of the VEST data repo, *"A dozen votes were reported countywide in King County and Yakima County, these were distributed by candidate to precincts based on their share of the precinct-level reported vote."* This means we will need to load in Kings County and Yakima County data separately in order to validate those two counties

**WA King County election results** is not stored on a precinct level in the SoS election results file, so we found King County's eCanvass on their [county data website](https://data.kingcounty.gov/Voting-Elections/2018-General-Election-eCanvass/ghxg-x8xz)
In order to download the data, click on the *Export* tab, and then click on **CSV**
The file will be saved as *2018_General_Election_eCanvass.csv*

**WA Yakima County election results** is stored on a precinct level in the SoS election results file, however in order to validate the distributed votes as per VEST's documentation, we downloaded their data from [county data website](https://results.vote.wa.gov/results/20181106/yakima/)
In order to download the data, right click on the **Precinct CSV** tab under *Export Results* tab and save as *.csv*
The file will be saved as *20181106_yakimaprecincts.csv*

King County precincts' IDs in the *2018_General_Election_eCanvass.csv* are not compatible with *20181106_allstateprecincts.csv* and therefore a name conversion dictionary was required and found on King County's [data repo](https://gis-kingcounty.opendata.arcgis.com/datasets/2018-voting-districts-for-king-county-votdst-area-2018/data?geometry=-122.297%2C47.651%2C-121.858%2C47.732)
In order to download the data, click on the **Download** tab and select **Spreadsheet** under *Full Dataset*.
The file will be saved as *2018_Voting_Districts_for_King_County___votdst_area_2018.csv*

In [43]:
wa_election_results = pd.read_csv('raw-from-source/20181106_allstateprecincts.csv')
wa_king = pd.read_csv('raw-from-source/2018_General_Election_eCanvass.csv')
wa_kings_names = pd.read_csv('raw-from-source/2018_Voting_Districts_for_King_County___votdst_area_2018.csv')
wa_yakima = pd.read_csv('raw-from-source/20181106_yakimaprecincts.csv')
wa_vest = gp.read_file("raw-from-source/wa_2018/wa_2018.shp")
wa_geo = gp.read_file("raw-from-source/2018Precincts_VERIFIED/2018Precincts_VERIFIED.shp")

In [23]:
display(wa_vest)

,COUNTYCD,PRCCODE,FULLPRC,G18USSDCAN,G18USSRHUT,geometry
0,AD,111,AD00000111,38,89,"POLYGON ((-118.37353 47.13614, -118.37353 47.1..."
1,AD,112,AD00000112,36,73,"POLYGON ((-118.37079 47.13292, -118.37098 47.1..."
2,AD,113,AD00000113,59,123,"POLYGON ((-118.36870 47.12700, -118.36866 47.1..."
3,AD,114,AD00000114,58,111,"POLYGON ((-118.37378 47.12402, -118.37362 47.1..."
4,AD,115,AD00000115,55,124,"POLYGON ((-118.37107 47.12031, -118.37097 47.1..."
...,...,...,...,...,...,...
7307,PI,28-505,PI00028505,578,357,"POLYGON ((-122.55416 47.24650, -122.55412 47.2..."
7308,PI,28-515,PI00028515,545,401,"POLYGON ((-122.56108 47.24406, -122.56119 47.2..."
7309,SN,21014030,SN21014030,52,66,"MULTIPOLYGON (((-122.38474 48.24150, -122.3847..."
7310,SN,21014002,SN21014002,74,120,"POLYGON ((-122.37703 48.23797, -122.37693 48.2..."


The final file contains 7 columns: 

**Unique ID** is a numeric sequence of IDs attributed to each entry, irrelevant to our purposes

**COUNTYCD** is a two-letter sequence abbreviating the county name

**PRCCODE** is a numeric (sometimes dash included) sequence denoting the unique code associated with a precinct within a county

**FULLPRC** is a 10-character (2 letters followed by 8 numbers) sequence denoting the unique PRC Code associated with each precinct on a statewide level

**G18USSDCAN** is a number of votes assigned to *Maria Cantwell*, democratic senate candidate

**G18USSRHUT** is a number of votes assigned to *Susan Hutchison*, republican senate candidate

**geometry** is shapefile data

Since **FULLPRC** is the only unique sequence (besides the Unique ID field which is selected randomly), we will use that field to merge and compare SoS data with VEST data for our validation purposes

# Cleaning Data

[The SoS website](https://results.vote.wa.gov/results/20181106/us-senator.html) reports that in 2018, Maria Cantwell who prefered Democratic, received 1,803,364 votes and Susan Hutchison who prefered Republican, received 1,282,804 votes

In [24]:
DEM_VOTE = 1803364
REP_VOTE = 1282804

Finding the number of votes lost to obfuscation

In [25]:
wa_senators = wa_election_results[wa_election_results["Race"] == "U.S. Senator"]
county_level = wa_senators[(wa_senators["PrecinctCode"] != -1)] ## PrecinctCode of -1 denotes total values
kings = wa_senators[wa_senators["CountyCode"] == "KI"] ## Kings only has total values
frames = [county_level, kings]
non_total = pd.concat(frames)
total_votes_reported = non_total["Votes"].sum()
total_votes_sos = REP_VOTE + DEM_VOTE
diff = abs(total_votes_reported - total_votes_sos)
print("Precinct level data has a reported total of votes of", total_votes_reported, "while the expected total of votes is", total_votes_sos)
print("resulting in", diff, "difference in votes due to obfuscation")

Precinct level data has a reported total of votes of 3084785 while the expected total of votes is 3086168
resulting in 1383 difference in votes due to obfuscation


Creating PrecinctCode column for counties who don't follow the standard 8-digit PRCCODE format

In [26]:
#FR
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "FR", non_total["PrecinctCode"].astype('str').str.rjust(3, '0'),
                                    non_total["PrecinctCode"])
#GY
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "GY", non_total["PrecinctCode"].astype('str').str.rjust(3, '0'),
                                    non_total["PrecinctCode"])
#TH
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "TH", non_total["PrecinctCode"].astype('str').str.rjust(3, '0'),
                                    non_total["PrecinctCode"])
#KP
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "KP", non_total["PrecinctCode"].astype('str').str.lstrip('1'),
                                    non_total["PrecinctCode"])
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "KP", non_total["PrecinctCode"].astype('str').str.lstrip('0'),
                                    non_total["PrecinctCode"])
#PI
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "PI", non_total["PrecinctCode"].astype('str').str.rjust(5, '0'),
                                    non_total["PrecinctCode"])
non_total["PrecinctCode"] = np.where(non_total["CountyCode"] == "PI", 
                                     non_total["PrecinctCode"].astype('str').str.slice(0, 2) + non_total["PrecinctCode"].astype('str').str.slice(2).str.rjust(3, '0'), 
                                     non_total["PrecinctCode"])

print(total_votes_reported)
print(non_total.Votes.sum())
display(non_total)

3084785
3084785


,Race,CountyCode,Candidate,PrecinctName,PrecinctCode,Votes
2,U.S. Senator,AD,Maria Cantwell,Ritzville Ward #1,111,38
3,U.S. Senator,AD,Susan Hutchison,Ritzville Ward #1,111,89
4,U.S. Senator,AD,Maria Cantwell,Ritzville Ward #2,112,36
5,U.S. Senator,AD,Susan Hutchison,Ritzville Ward #2,112,73
6,U.S. Senator,AD,Maria Cantwell,Ritzville Ward #3,113,59
...,...,...,...,...,...,...
9395,U.S. Senator,YA,Susan Hutchison,5101.38,5101,174
9396,U.S. Senator,YA,Maria Cantwell,5202.473(*),5202,0
9397,U.S. Senator,YA,Susan Hutchison,5202.473(*),5202,0
2496,U.S. Senator,KI,Maria Cantwell,Total,-1,708654


Processing King county, using their data to create precinct level rows

In [28]:
ki = non_total[non_total["CountyCode"] == "KI"]
rest = non_total[non_total["CountyCode"] != "KI"]
wa_king = wa_king[wa_king["Race"] == "US Senator"]
wa_king_frames = [wa_king[wa_king["Party"] == "Rep"], wa_king[wa_king["Party"] == "Dem"]]
wa_king = pd.concat(wa_king_frames)
display(wa_king)


,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
210936,SEA 34-1533,US Senator,34,8,7,Total,Rep,Susan Hutchison,26
211335,SEA 34-1534,US Senator,34,8,7,Total,Rep,Susan Hutchison,69
211783,BEL 48-0156,US Senator,48,6,9,Total,Rep,Susan Hutchison,172
212182,SEA 34-1535,US Senator,34,8,7,Total,Rep,Susan Hutchison,45
212630,SEA 34-1536,US Senator,34,8,7,Total,Rep,Susan Hutchison,39
...,...,...,...,...,...,...,...,...,...
370497,SEA 34-1507,US Senator,34,8,7,Total,Dem,Maria Cantwell,249
370504,SEA 34-1508,US Senator,34,8,7,Total,Dem,Maria Cantwell,166
370511,SEA 34-1509,US Senator,34,8,7,Total,Dem,Maria Cantwell,202
370518,SEA 34-1510,US Senator,34,8,7,Total,Dem,Maria Cantwell,303


renaming King county precinct's to match with SoS data

In [29]:
wa_kings_names = pd.read_csv('raw-from-source/2018_Voting_Districts_for_King_County___votdst_area_2018.csv')
wa_kings_names["votdst"] = wa_kings_names["votdst"].astype('str').str.rjust(4, '0')
wa_kings_names = {name: num for name, num in zip(wa_kings_names["NAME"], wa_kings_names['votdst'])}
wa_king["Precinct"] = wa_king["Precinct"].apply(lambda x: wa_kings_names[x] if x in wa_kings_names else x)
#finding votes not associated with any precinct in king
display(wa_king[(wa_king["Precinct"] == "ELECTIONS OFFICE") & (wa_king["Race"] == 'US Senator') &
               ((wa_king["CounterType"] == "Maria Cantwell") | (wa_king["CounterType"] == "Susan Hutchison"))])

elections_office = wa_king[(wa_king["Precinct"] == "ELECTIONS OFFICE") & (wa_king["Race"] == 'US Senator') &
               ((wa_king["CounterType"] == "Maria Cantwell") | (wa_king["CounterType"] == "Susan Hutchison"))]

display(wa_king)

wa_king["FULLPRC"] = "KI" + wa_king["Precinct"].apply(lambda x:str(x).zfill(8))
wa_king = wa_king.pivot_table(index="FULLPRC", columns="CounterType", values="SumOfCount", aggfunc="first").reset_index()
display(wa_king)

,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
364791,ELECTIONS OFFICE,US Senator,0,0,9,Total,Rep,Susan Hutchison,1
364792,ELECTIONS OFFICE,US Senator,0,0,9,Total,Dem,Maria Cantwell,10


,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
210936,1533,US Senator,34,8,7,Total,Rep,Susan Hutchison,26
211335,1534,US Senator,34,8,7,Total,Rep,Susan Hutchison,69
211783,0156,US Senator,48,6,9,Total,Rep,Susan Hutchison,172
212182,1535,US Senator,34,8,7,Total,Rep,Susan Hutchison,45
212630,1536,US Senator,34,8,7,Total,Rep,Susan Hutchison,39
...,...,...,...,...,...,...,...,...,...
370497,1507,US Senator,34,8,7,Total,Dem,Maria Cantwell,249
370504,1508,US Senator,34,8,7,Total,Dem,Maria Cantwell,166
370511,1509,US Senator,34,8,7,Total,Dem,Maria Cantwell,202
370518,1510,US Senator,34,8,7,Total,Dem,Maria Cantwell,303


CounterType,FULLPRC,Maria Cantwell,Susan Hutchison
0,KI00000001,258,58
1,KI00000003,265,111
2,KI00000009,113,64
3,KI00000010,483,258
4,KI00000011,140,56
...,...,...,...
2599,KI00003798,297,44
2600,KI00003799,140,100
2601,KI00003800,271,202
2602,KI00003801,214,93


distributing the 11 votes from Elections Office which are reported on county level to precincts with highest amount of voters for that specific party

In [30]:
rep_vote = elections_office.iloc[0]['SumOfCount']
dem_vote = elections_office.iloc[1]['SumOfCount']
print(dem_vote + rep_vote)

rep_total = wa_king["Susan Hutchison"].to_numpy()
dem_total = wa_king["Maria Cantwell"].to_numpy()


rep = np.argsort(rep_total)[::-1][:rep_vote]
dem = np.argsort(dem_total)[::-1][:dem_vote]

for i in dem:
    wa_king.iloc[i, 1] +=1
    
for i in rep:
    wa_king.iloc[i, 2] +=1
    
wa_king_final = wa_king[wa_king["FULLPRC"] != "KIELECTIONS OFFICE"]
display(wa_king_final)

11


CounterType,FULLPRC,Maria Cantwell,Susan Hutchison
0,KI00000001,258,58
1,KI00000003,265,111
2,KI00000009,113,64
3,KI00000010,483,258
4,KI00000011,140,56
...,...,...,...
2598,KI00003797,292,38
2599,KI00003798,297,44
2600,KI00003799,140,100
2601,KI00003800,271,202


Use Yakima data to find any county level votes as reported in VEST documentation

In [35]:
ya = rest[rest["CountyCode"] == "YA"]
display(ya)
print(ya['Votes'].sum())
wa_yakima.rename(columns={0:'Race',1:'Candidate',2:'PrecinctName',3:'PrecinctCode',4:'Votes'},inplace=True)
display(wa_yakima)

yakima_clean = wa_yakima[wa_yakima['Race'] == "U.S. Senator"]
yakima_clean = yakima_clean.iloc[2:]
display(yakima_clean)
yakima_clean['Votes'] = yakima_clean['Votes'].astype(int)
print(yakima_clean["Votes"].sum())

,Race,CountyCode,Candidate,PrecinctName,PrecinctCode,Votes
9052,U.S. Senator,YA,Maria Cantwell,0001 - COUNTY COURTHOUSE(*),1,0
9053,U.S. Senator,YA,Susan Hutchison,0001 - COUNTY COURTHOUSE(*),1,0
9054,U.S. Senator,YA,Maria Cantwell,0101.333,101,130
9055,U.S. Senator,YA,Susan Hutchison,0101.333,101,102
9056,U.S. Senator,YA,Maria Cantwell,0102.333,102,94
...,...,...,...,...,...,...
9393,U.S. Senator,YA,Susan Hutchison,5020.584,5020,262
9394,U.S. Senator,YA,Maria Cantwell,5101.38,5101,96
9395,U.S. Senator,YA,Susan Hutchison,5101.38,5101,174
9396,U.S. Senator,YA,Maria Cantwell,5202.473(*),5202,0


70412


,U.S. Senator,Maria Cantwell,Total,-1,29476
0,U.S. Senator,Susan Hutchison,Total,-1,40958
1,U.S. Senator,Maria Cantwell,0001 - COUNTY COURTHOUSE(*),1,0
2,U.S. Senator,Susan Hutchison,0001 - COUNTY COURTHOUSE(*),1,0
3,U.S. Senator,Maria Cantwell,0101.333,101,130
4,U.S. Senator,Susan Hutchison,0101.333,101,102
...,...,...,...,...,...
7104,Initiative to the Legislature 940 Initiative M...,No,5020.584,5020,239
7105,Initiative to the Legislature 940 Initiative M...,Yes,5101.38,5101,114
7106,Initiative to the Legislature 940 Initiative M...,No,5101.38,5101,163
7107,Initiative to the Legislature 940 Initiative M...,Yes,5202.473(*),5202,0


KeyError: 'Race'

get both non kings county and kings county tables ready for concat by renaming their columns

In [49]:
pivoted = rest.pivot_table(columns = ["Candidate"], index = ["PrecinctCode", "CountyCode"], values = ["Votes"], aggfunc='first').reset_index()
pivoted['PrecinctCode'] = pivoted['PrecinctCode'].apply(lambda x:str(x).zfill(8))
pivoted['FULLPRC'] = pivoted["CountyCode"] + pivoted["PrecinctCode"]
pivoted.columns = pivoted.columns.get_level_values(0)
pivoted.columns = ["PRCCODE", "COUNTYCD", "G18USSDCAN", "G18USSRHUT", "FULLPRC" ]
pivoted.head()
wa_king_final.rename(columns = {'Maria Cantwell':'G18USSDCAN','Susan Hutchison':'G18USSRHUT'},inplace=True)
wa_king_final["COUNTYCD"] = "KI"
wa_king_final["PRCCODE"] = wa_king_final.FULLPRC.str[2:]
wa_king_final.head()

CounterType,FULLPRC,G18USSDCAN,G18USSRHUT,COUNTYCD,PRCCODE
0,KI00000001,258,58,KI,00000001
1,KI00000003,265,111,KI,00000003
2,KI00000009,113,64,KI,00000009
3,KI00000010,483,258,KI,00000010
4,KI00000011,140,56,KI,00000011


In [50]:
concat = pd.concat([pivoted,wa_king_final],ignore_index=True)
concat.head()

,PRCCODE,COUNTYCD,G18USSDCAN,G18USSRHUT,FULLPRC
0,00000001,AS,57,138,AS00000001
1,00000001,CU,18,65,CU00000001
2,00000001,CZ,260,219,CZ00000001
3,00000001,GR,107,257,GR00000001
4,00000001,KS,85,170,KS00000001


# Merge election and shapefiles

In [54]:
wa_geo.columns = ["COUNTYCD", "PRCCODE", "FULLPRC", "geometry"]
display(wa_geo)
final_file = wa_geo.merge(concat, on="FULLPRC", how='right')
final_file.head()

,COUNTYCD,PRCCODE,FULLPRC,geometry
0,AD,111,AD00000111,"POLYGON ((-118.37353 47.13614, -118.37353 47.1..."
1,AD,112,AD00000112,"POLYGON ((-118.37079 47.13292, -118.37098 47.1..."
2,AD,113,AD00000113,"POLYGON ((-118.36870 47.12700, -118.36866 47.1..."
3,AD,114,AD00000114,"POLYGON ((-118.37378 47.12402, -118.37362 47.1..."
4,AD,115,AD00000115,"POLYGON ((-118.37107 47.12031, -118.37097 47.1..."
...,...,...,...,...
7331,CZ,72,CZ00000072,"POLYGON ((-122.74414 45.91787, -122.74471 45.9..."
7332,CZ,73,CZ00000073,"POLYGON ((-122.76049 45.93007, -122.75845 45.9..."
7333,CZ,74,CZ00000074,"POLYGON ((-122.24605 46.12817, -122.24597 46.1..."
7334,CZ,8,CZ00000008,"POLYGON ((-122.79423 46.18476, -122.79437 46.1..."


,COUNTYCD_x,PRCCODE_x,FULLPRC,geometry,PRCCODE_y,COUNTYCD_y,G18USSDCAN,G18USSRHUT
0,AS,1,AS00000001,"MULTIPOLYGON (((-117.27328 46.03418, -117.2727...",00000001,AS,57,138
1,CU,1,CU00000001,"POLYGON ((-118.11678 46.45570, -118.11632 46.4...",00000001,CU,18,65
2,CZ,1,CZ00000001,"POLYGON ((-122.93611 46.11138, -122.93611 46.1...",00000001,CZ,260,219
3,GR,1,GR00000001,"POLYGON ((-119.29929 47.17090, -119.29930 47.1...",00000001,GR,107,257
4,KS,1,KS00000001,"MULTIPOLYGON (((-120.92971 47.20218, -120.9295...",00000001,KS,85,170


In [57]:
display(final_file[~final_file["FULLPRC"].isin(wa_vest["FULLPRC"])])

print("vest")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(wa_vest[~wa_vest["FULLPRC"].isin(final_file["FULLPRC"])])

,COUNTYCD_x,PRCCODE_x,FULLPRC,geometry,PRCCODE_y,COUNTYCD_y,G18USSDCAN,G18USSRHUT
14,NaN,NaN,YA00000001,None,00000001,YA,0,0
49,OK,4,OK00000004,"POLYGON ((-118.83685 48.95986, -118.83683 48.9...",00000004,OK,7,15
111,OK,9,OK00000009,"POLYGON ((-119.05413 48.91751, -119.05426 48.9...",00000009,OK,18,41
203,OK,17,OK00000017,"POLYGON ((-119.55190 48.38314, -119.55312 48.3...",00000017,OK,25,33
289,OK,25,OK00000025,"POLYGON ((-119.71170 48.20768, -119.71117 48.2...",00000025,OK,22,92
344,OK,31,OK00000031,"POLYGON ((-119.60390 48.61248, -119.59033 48.6...",00000031,OK,0,0
362,OK,33,OK00000033,"POLYGON ((-119.46682 48.62612, -119.46594 48.6...",00000033,OK,23,88
371,OK,34,OK00000034,"POLYGON ((-119.00111 48.58300, -119.00092 48.5...",00000034,OK,53,70
427,OK,41,OK00000041,"POLYGON ((-118.83603 48.66819, -118.83621 48.6...",00000041,OK,45,118
500,OK,50,OK00000050,"POLYGON ((-119.59033 48.61236, -119.58197 48.6...",00000050,OK,27,79


vest


,COUNTYCD,PRCCODE,FULLPRC,G18USSDCAN,G18USSRHUT,geometry
68,BE,1291,BE00001291,0,0,"POLYGON ((-119.63928 46.40417, -119.62353 46.4..."
69,BE,1292,BE00001292,0,0,"POLYGON ((-119.87430 46.47642, -119.86441 46.4..."
73,BE,1364,BE00001364,0,0,"POLYGON ((-119.37075 46.07179, -119.37076 46.0..."
80,BE,1407,BE00001407,0,0,"POLYGON ((-119.33748 46.32104, -119.33693 46.3..."
121,BE,2409,BE00002409,0,0,"POLYGON ((-119.61858 46.37505, -119.58059 46.3..."
140,BE,2632,BE00002632,0,0,"POLYGON ((-119.19069 46.17014, -119.18024 46.1..."
152,BE,3003,BE00003003,0,0,"POLYGON ((-119.47406 46.25115, -119.47145 46.2..."
294,BE,6316,BE00006316,0,0,"POLYGON ((-119.35224 46.22573, -119.35307 46.2..."
296,BE,96,BE00000096,0,0,"POLYGON ((-119.78873 46.49076, -119.78906 46.5..."
297,BE,97,BE00000097,0,0,"POLYGON ((-119.42803 46.38269, -119.42803 46.3..."


In [58]:
final_merge = wa_vest.merge(final_file, on="FULLPRC", how='outer')
final_merge.head()

,COUNTYCD,PRCCODE,FULLPRC,G18USSDCAN_x,G18USSRHUT_x,geometry_x,COUNTYCD_x,PRCCODE_x,geometry_y,PRCCODE_y,COUNTYCD_y,G18USSDCAN_y,G18USSRHUT_y
0,AD,111,AD00000111,38.0,89.0,"POLYGON ((-118.37353 47.13614, -118.37353 47.1...",AD,111,"POLYGON ((-118.37353 47.13614, -118.37353 47.1...",00000111,AD,38.0,89.0
1,AD,112,AD00000112,36.0,73.0,"POLYGON ((-118.37079 47.13292, -118.37098 47.1...",AD,112,"POLYGON ((-118.37079 47.13292, -118.37098 47.1...",00000112,AD,36.0,73.0
2,AD,113,AD00000113,59.0,123.0,"POLYGON ((-118.36870 47.12700, -118.36866 47.1...",AD,113,"POLYGON ((-118.36870 47.12700, -118.36866 47.1...",00000113,AD,59.0,123.0
3,AD,114,AD00000114,58.0,111.0,"POLYGON ((-118.37378 47.12402, -118.37362 47.1...",AD,114,"POLYGON ((-118.37378 47.12402, -118.37362 47.1...",00000114,AD,58.0,111.0
4,AD,115,AD00000115,55.0,124.0,"POLYGON ((-118.37107 47.12031, -118.37097 47.1...",AD,115,"POLYGON ((-118.37107 47.12031, -118.37097 47.1...",00000115,AD,55.0,124.0


In [60]:
# Let's try to verify on a county wide basis 
def validater_row_vect(df, name_column, column_list, verbose=False):
    matching_rows = 0
    different_rows = 0
    
    county_join_cols = (df[name_column]).to_numpy()
    
    # it's because before, we were checking that a row was entirely consistent
    # here, we are double checking a lot
    # so, let's just keep one track of the rows that are messed up
    
    wrong_idxs = np.zeros(len(df))
    for i in column_list:
        left_data = df[i + "_x"].to_numpy()
        right_data = df[i + "_y"].to_numpy()
        
        local_idxs = np.where(left_data != right_data)
        wrong_idxs[local_idxs] = 1
        #print("Wrong idxs", wrong_idxs)
        
    # we are close, we get the same result, but are double adding lots of rows
    different_rows += np.sum(wrong_idxs)
    matching_rows += len(df) - different_rows
    
    diff_list = county_join_cols[np.where(wrong_idxs == 1)]
    diff_counties = list(set([county[:2] for county in diff_list]))
    
    if int(different_rows) != 0 or verbose:
        print("There are ", len(df.index)," total rows")
        print(f"{int(different_rows)} of these rows have election result differences")
        print(f"{int(matching_rows)} of these rows are the same")
        print(diff_list)
        # print(diff_counties)
        print("")
        
    return (int(different_rows) == 0, diff_list)
    
    
counties = final_merge["COUNTYCD_x"].unique()
nonvalid = []
valid = []
obfuscated_counties = 
for county in counties:
    print(county)
    curr = final_merge[final_merge["COUNTYCD_x"] == county]
    match, nonmatch = validater_row_vect(curr, "FULLPRC", ["G18USSDCAN", "G18USSRHUT"])
    if match:
        valid.append(county)
    else:
        nonvalid.append(county)

print("Match exactly", valid)
print("Need to check for obfuscation", nonvalid)

AD
AS
BE
There are  231  total rows
16 of these rows have election result differences
215 of these rows are the same
['BE00001020' 'BE00001021' 'BE00001060' 'BE00001061' 'BE00001090'
 'BE00001241' 'BE00001360' 'BE00001362' 'BE00001363' 'BE00001403'
 'BE00001408' 'BE00001413' 'BE00005631' 'BE00005634' 'BE00005636'
 'BE00006311']

nan
CH
There are  85  total rows
3 of these rows have election result differences
82 of these rows are the same
['CH00001158' 'CH00000136' 'CH00000152']

CM
CR
There are  287  total rows
3 of these rows have election result differences
284 of these rows are the same
['CR00000511' 'CR00000513' 'CR00000576']

CU
DG
FE
FR
There are  104  total rows
3 of these rows have election result differences
101 of these rows are the same
['FR00000003' 'FR00000068' 'FR00000091']

GA
There are  12  total rows
2 of these rows have election result differences
10 of these rows are the same
['GA00000101' 'GA00000102']

GR
GY
There are  75  total rows
2 of these rows have election 

In [74]:
def validate_total(vest, county, county_name):
    print("validating", county_name)
    total_dem = county[county["Candidate"] == "Maria Cantwell"]["Votes"].iloc[0]
    total_rep = county[county["Candidate"] == "Susan Hutchison"]["Votes"].iloc[0]
        
    vest_dem = vest[vest["COUNTYCD_x"] == county_name]["G18USSDCAN_x"].iloc[0]
    vest_rep = vest[vest["COUNTYCD_x"] == county_name]["G18USSRHUT_x"].iloc[0]
    
    if total_dem == vest_dem and total_rep == vest_rep:
        print(county_name, "validated")
    else:
        print(county_name, "has a difference in votes", total_dem, vest_dem, total_rep, vest_rep)
        
for county in nonvalid:
    county_table = rest[rest["CountyCode"] == county]
    try:
        validate_total(final_merge, county_table, county)
    except AssertionError:
        continue

validating BE
BE has a difference in votes 201 0.0 261 3.0
validating CH
CH validated
validating CR
CR validated
validating FR
FR validated
validating GA
GA has a difference in votes 0 0.0 0 2.0
validating GY
GY has a difference in votes 205 63.0 19 111.0
validating LE
LE validated
validating OK
OK validated
validating PI
PI validated
validating SK
SK validated
validating SN
SN validated
validating TH
TH validated
validating WL
WL validated
validating WM
WM has a difference in votes 558 272.0 194 430.0
validating WT
WT has a difference in votes 124 23.0 208 71.0
validating YA
YA has a difference in votes 0 130.0 0 102.0
validating CZ
CZ has a difference in votes 260 252.0 219 249.0


validating geometry

In [76]:
def validate_geom(gdf1,gdf2,sort_field,proj_epsg,dec=0):
    gdf1["geometry"]=gdf1.buffer(0) #buffer by 0 to remove any non polygons
    gdf2["geometry"]=gdf2.buffer(0)
    proj = gdf1.crs #project to same projection
    gdf2 = gdf2.to_crs(proj)
    gdf1 = gdf1.to_crs(proj)
    gdf1 = gdf1.sort_values(by=[sort_field]) #sort values by the unique field
    gdf2 = gdf2.sort_values(by=[sort_field])
    gdf2.reset_index(drop=True,inplace=True) #drop the index
    gdf1.reset_index(drop=True,inplace=True)
    ###Add in print link difference between the two files
    ## Look at GA example
    #see the difference between the precinct and final file
    ngdf1 = gdf1.to_crs(epsg=proj_epsg)
    ngdf2 = gdf2.to_crs(epsg=proj_epsg)
    ngdf1 = ngdf1.buffer(0)
    ngdf2 = ngdf2.buffer(0)
    file = ngdf2.difference(ngdf1)
    print('Difference between gdf1 and gdf2 as a percent of gdf1 total area ', str((sum(file.area)/sum(ngdf1.area))*100))
    print('Difference between gdf1 and gdf2 as a percent of gdf2 total area ', str((sum(file.area)/sum(ngdf2.area))*100))
    #To 6 decimal places, the two files are equal 
    t_f_geom = list(gdf1.geom_almost_equals(gdf2,decimal=dec))
    gdf1['GEOM_EQUALS'] = t_f_geom
    gdf2['GEOM_EQUALS'] = t_f_geom
    unique_geom = []
    for i in t_f_geom:
        if i not in unique_geom:
            unique_geom.append(i)
    for i in unique_geom:
        percent = str((t_f_geom.count(i)/len(t_f_geom))*100) 
        print(str(t_f_geom.count(i)) + ' wards are ' + str(i) + ' which is ' + percent + ' percent of all wards.')
    return gdf1, gdf2



In [77]:
geom_validate1 = validate_geom(wa_vest,wa_geo,'FULLPRC',32148)


/Users/izzy/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


Difference between gdf1 and gdf2 as a percent of gdf1 total area  nan
Difference between gdf1 and gdf2 as a percent of gdf2 total area  nan


/Users/izzy/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


ValueError: Length of values (7336) does not match length of index (7312)

In [ ]:
wa_vest = wa_vest[wa_vest['COUNTYCD']!='OK'].copy()
wa_geo = wa_geo[wa_geo['CountyCd']!='OK'].copy()

In [ ]:
geom_validate2 = validate_geom(wa_vest,wa_geo,'FULLPRC',32148)

wa_geo = gp.read_file("raw-from-source/2018Precincts_OK_Consol/2018Precincts_OK_Consol.shp")
wa_vest = gp.read_file("raw-from-source/wa_2018/wa_2018.shp")

In [ ]:
wa_kings_names = pd.read_csv('raw-from-source/2018_Voting_Districts_for_King_County___votdst_area_2018.csv')
wa_kings_names["votdst"] = wa_kings_names["votdst"].astype('str').str.rjust(4, '0')
wa_kings_names = {name: num for name, num in zip(wa_kings_names["NAME"], wa_kings_names['votdst'])}
wa_king["Precinct"] = wa_king["Precinct"].apply(lambda x: wa_kings_names[x] if x in wa_kings_names else x)
#finding votes not associated with any precinct in king
display(wa_king[(wa_king["Precinct"] == "ELECTIONS OFFICE") & (wa_king["Race"] == 'US Senator') &
               ((wa_king["CounterType"] == "Maria Cantwell") | (wa_king["CounterType"] == "Susan Hutchison"))])

elections_office = wa_king[(wa_king["Precinct"] == "ELECTIONS OFFICE") & (wa_king["Race"] == 'US Senator') &
               ((wa_king["CounterType"] == "Maria Cantwell") | (wa_king["CounterType"] == "Susan Hutchison"))]

display(wa_king)

wa_king["FULLPRC"] = "KI" + wa_king["Precinct"].apply(lambda x:str(x).zfill(8))
wa_king = wa_king.pivot_table(index="FULLPRC", columns="CounterType", values="SumOfCount", aggfunc="first").reset_index()
display(wa_king)